In [2]:
# school_attendance_etl.py
import pandas as pd
from datetime import datetime
import os

def generate_sample_data():
    database_file = 'school_attendance.csv'
    schools = ['01M015', '02M394', '03K403', '04M409', '05M280']
    data = []
    start_date = datetime(2018, 9, 1)

    for i in range(1, 91):  # 90 days of data
            date = start_date + timedelta(days=i)
            if date.weekday() >= 5:  # Skip weekends
                continue
                
            for school in schools:
                enrolled = random.randint(150, 200)
                absent = random.randint(5, 30)
                present = enrolled - absent
                data.append({
                    'School DBN': school,
                    'Date': date.date().strftime('%Y%m%d'),
                    'Enrolled': enrolled,
                    'Absent': absent,
                    'Present': present,
                    'Released': 0,
                    'last_updated': (date + timedelta(hours=random.randint(0, 23))).isoformat()
                })

    df = pd.DataFrame(data)
    df.to_csv('school_attendance.csv', index=False)
print("Sample data generated in school_attendance.csv")

#FULL EXTRACTION
try:
    df = pd.read_csv(
        "school_attendance.csv",
        parse_dates=["last_updated"],
        converters={"Date": lambda x: datetime.strptime(x, "%Y%m%d")}
    )
    print(f" Full extraction: Loaded ALL {len(df)} records.")
    # You can assign to a variable if needed, e.g. df_full = df
except Exception as e:
    print(f" Full extraction failed: {str(e)}")
    df = None  # or df_full = None

#INCREMENTAL EXTRACTION
def incremental_extraction(max_rows=50):
    """Perform incremental extraction with row limit"""
    try:
        # 1. Handle last_extraction.txt
        if not os.path.exists("last_extraction.txt"):
            with open("last_extraction.txt", "w") as f:
                default_date = "2018-01-01T00:00:00"
                f.write(default_date)
            print(f"Created last_extraction.txt with default date {default_date}")
        
        with open("last_extraction.txt", "r") as f:
            last_extraction = f.read().strip()

        # 2. Load and process data
        df = pd.read_csv("school_attendance.csv", 
                        parse_dates=["last_updated"],
                        converters={'Date': lambda x: datetime.strptime(x, '%Y%m%d')})
        
        last_extraction_time = pd.to_datetime(last_extraction)
        df_incremental = df[df['last_updated'] > last_extraction_time]
        
        # 3. Apply row limit
        df_limited = df_incremental.head(max_rows)
        
        print(f"\nFound {len(df_incremental)} new records since {last_extraction}")
        print(f"Returning {len(df_limited)} records (max {max_rows})")
        
        if len(df_limited) > 0:
            print("\nSample of returned records:")
            print(df_limited.head())
            
            # 4. Update timestamp
            new_checkpoint = df_limited['last_updated'].max()
            with open("last_extraction.txt", "w") as f:
                f.write(new_checkpoint.isoformat())
            print(f"\nUpdated last_extraction.txt to {new_checkpoint}")
            
            return df_limited
        else:
            print("\nNo new records found.")
            return pd.DataFrame()  # Return empty DataFrame if no new records
            
    except Exception as e:
        print(f"\nError: {str(e)}")
        return None

if __name__ == "__main__":
    # Generate sample data if needed
    generate_sample_data()
    
    # Run incremental extraction (gets max 50 rows)
    result = incremental_extraction(max_rows=50)
    
    # Optional: Save the extracted data to a new file
    if result is not None and not result.empty:
        result.to_csv("latest_extracted_records.csv", index=False)
        print("\nSaved extracted records to latest_extracted_records.csv")

ModuleNotFoundError: No module named 'pandas'